# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI(base_url='https://api.xiaoai.plus/v1')

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示的是一条位于高草地中的木制栈道，引领视线穿越翠绿的自然景观直至远方。草地上没有明显的花卉，但不同色调的绿色给人一种生机勃勃的感觉。天空晴朗，布满了绵延的白云，为整个景色增添了宽广与深远的感觉。这种场景通常见于公园或自然保护区，是人们散步或观察自然生态的理想场所。整体上，这幅图片传达了一种宁静与和谐的氛围。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图展示的是一条位于高草地中的木制栈道，引领视线穿越翠绿的自然景观直至远方。草地上没有明显的花卉，但不同色调的绿色给人一种生机勃勃的感觉。天空晴朗，布满了绵延的白云，为整个景色增添了宽广与深远的感觉。这种场景通常见于公园或自然保护区，是人们散步或观察自然生态的理想场所。整体上，这幅图片传达了一种宁静与和谐的氛围。'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(base_url='https://api.xiaoai.plus/v1')  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [9]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张幽默的比较图，展示了一只肌肉发达的沙皮犬和一只普通体型的沙皮犬。左边的肌肉沙皮犬被标记为“16岁的我”上方的描述为“工作后的我”，给人一种幻想中年轻时充满力量和活力的模样。右边的普通沙皮犬则给人一种无奈、颓废的感觉，下方的描述是工作压力和生活的不易。整幅图通过夸张的对比，幽默地表达了人们对于年轻时与工作后状态的想象与现实之间的差距。这种类型的图通常用来在社交媒体上引发共鸣和笑声。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [12]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(base_url='https://api.xiaoai.plus/v1')  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [13]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图表展示了从1980年到2020年间，美国、中国、日本和德国的国内生产总值(GDP)的比较。图中将GDP数值以万亿美元表示，并使用不同的颜色代表不同的国家：蓝色代表美国，红色代表中国，紫色代表日本，绿色代表德国。

从图中可以看到，美国的GDP整体呈稳步上升趋势，尤其是从1995年以后增长更为显著。中国的GDP在1990年之前增长相对缓慢，但从1990年开始快速增长，并在2010年之后增长更为迅猛。至2020年，中国的GDP已接近美国。

日本的GDP在1980年代和1990年代初期有较快增长，但自1995年后增长放缓，并在2000年以后基本保持平稳。德国的GDP增长则相对平稳，整体增长幅度较小。

总的来看，这张图表清晰地展示了这四个大国在过去四十年间的经济增长情况，并突出了中国经济的快速增长，以及美国作为世界最大经济体的地位。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [14]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片是一本笔记本的一页，上面记载了一些关于深度学习模型调优技术的笔记。主要内容包括：

1. **Prompt Tuning （简单模型 Small Model）**：
   - 提到了输入 \( X = [X_1, X_2, ..., X_n] \) 的表示，这是通过嵌入（Embedding）技术将原始的数据标记转换成能够输入到模型中的形式。
   - \( Y = WX' \) 表示输出 \( Y \) 是通过输入 \( X' \) 与权重矩阵 \( W \) 的线性变换得到。

2. **Prefix Tuning**：
   - 提到了通过扩展权重 \( W' = [W_p; W] \) 来调整前缀，生成新的输出 \( Y \)。

3. **LoRA（低秩逼近 Low-Rank Approximation）技术**：
   - 记录了 \( Y = (W + \Delta W)X \) 公式，其中 \( \Delta W = AB \) 用于调整原有的权重矩阵 \( W \)，是一个低秩矩阵乘法形式（\( A \) 和 \( B \) 是低秩矩阵）。
   - 提到了将此技术应用于参数调优的过程和内存效率的提升。

最后，还有一些关于模型大小和存储需求的对比（如LoRA vs LaMA），以及相关的内存使用数据（LoRA - 78GB, LaMA - 65GB, QLoRA - 48GB）。这些信息表明作者正在研究或记录不同调优技术在不同参数和内存效率方面的表现。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [15]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片是一本笔记本的一页，上面记载了一些关于深度学习模型调优技术的笔记。主要内容包括：

1. **Prompt Tuning （简单模型 Small Model）**：
   - 提到了输入 \( X = [X_1, X_2, ..., X_n] \) 的表示，这是通过嵌入（Embedding）技术将原始的数据标记转换成能够输入到模型中的形式。
   - \( Y = WX' \) 表示输出 \( Y \) 是通过输入 \( X' \) 与权重矩阵 \( W \) 的线性变换得到。

2. **Prefix Tuning**：
   - 提到了通过扩展权重 \( W' = [W_p; W] \) 来调整前缀，生成新的输出 \( Y \)。

3. **LoRA（低秩逼近 Low-Rank Approximation）技术**：
   - 记录了 \( Y = (W + \Delta W)X \) 公式，其中 \( \Delta W = AB \) 用于调整原有的权重矩阵 \( W \)，是一个低秩矩阵乘法形式（\( A \) 和 \( B \) 是低秩矩阵）。
   - 提到了将此技术应用于参数调优的过程和内存效率的提升。

最后，还有一些关于模型大小和存储需求的对比（如LoRA vs LaMA），以及相关的内存使用数据（LoRA - 78GB, LaMA - 65GB, QLoRA - 48GB）。这些信息表明作者正在研究或记录不同调优技术在不同参数和内存效率方面的表现。

![](./images/handwriting_1.jpg)

In [16]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本上的手写笔记，内容涉及到机器学习领域中的一些技术和方法，特别是关于自然语言处理（NLP）中的“transformer”模型和其变体的细节。从笔记中可见，作者记录了关于transformer模型的各种调优方法和部分评估标准。

左侧的笔记提到：
- PEFT (可能指的是某种性能优化技术)
- SOTA（State of the Art的缩写，指领域内最先进的研究或技术）
- PBFT Methods
- Prompt Tuning（一种调整预训练模型以适应特定任务的方法）
  - Adaptor、Prefix、P-Tuning 手法的年份和来源也被记录，如"Adaptor (2019, Google)”。

右侧的笔记则记录了：
- 多模态指令式微调（multi-modality instruction finetuning）
- LoRA 和 PETC 等新方法
- Adapters的使用（这是一种插入到预训练模型的小模块，使得模型能够更好地适应新任务而不需要大规模重新训练）

此外，还涉及到优化技术如LoRA、GLoRA等的变体，以及不同技术如Scaled parallel adapter、MAM Adapters的性能对比和优化结果。

总的来说，这些笔记体现出作者在积极学习和记录目前自然语言处理领域里transformer模型调优和应用的各种先进技术和方法。这种笔记对于理解和跟进该领域的快速发展非常有帮助。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [18]:
## Homework 1
content = query_base64_image_description("./images/handwriting_2.png")
print(content)

这张图片中记录的是关于概率论和集合论的笔记。内容包括以下几部分：

1. **集合运算** 
    - ① 交换律（交换律适用于并集和交集）
    - ② 吸收律
    - ③ 互补运算定理

2. **集合运算的属性**
    - 补集的公式：如\(A' = U - A\)（全集减去某集合A表示A的补集）
    - 去补律：涉及到补集的运算
    - 幂等律：集合自身的并集或交集等于自身
    - 定律：如空集的定义等

3. **集合运算符号说明**
    - ① 小于或等于
    - ② 等于
    - ③ 属于
    - ④ 并集
    - ⑤ 交集
    - ⑥ 空
    - ⑦ 子集  

4. **更高级的集合操作和表示**
    - 例如集合的差集 \(A - B = AB' = A(AB')\)（表示A中不属于B的部分）
    - 泛化的集合运算公式，如无穷并、无穷交等。

5. **概率运算**
    - ① 事件发生的概率大于0
    - ② 确定性事件的概率为1
    - ③ 加法公式和一般加法公式，用于计算复合事件的概率

这些笔记深入介绍了数学中集合和概率的基础概念，符号，以及它们的运算规则，适用于进一步的数学学习和应用。
